In [2]:
import pandas as pd
import geopandas

# Part 1: Data Preprocessing
- Remove unnecessary columns
- Merge with countries geometry data

In [3]:
df = pd.read_csv("data.csv")
countries = geopandas.read_file("ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Alpha-2 code', 'Numeric code', 'Latitude (average)', 'Longitude (average)'])

- Make the country names corresponding in two datasets

In [4]:
df['Country'] = df['Country'].replace('United States', 'United States of America')
df['Country'] = df['Country'].replace('Hong Kong', 'Hong Kong S.A.R.')
df['Country'] = df['Country'].replace('Palestinian Territory, Occupied', 'Palestine')
df['Country'] = df['Country'].replace('Bahamas', 'The Bahamas')
df['Country'] = df['Country'].replace('Czech Republic', 'Czechia')
df['Country'] = df['Country'].replace('Russian Federation', 'Russia')
df['Country'] = df['Country'].replace('Korea, Republic of', 'South Korea')
df['Country'] = df['Country'].replace('Congo, the Democratic Republic of the', 'Democratic Republic of the Congo')
df['Country'] = df['Country'].replace('Venezuela, Bolivarian Republic of', 'Venezuela')
df['Country'] = df['Country'].replace('Iran, Islamic Republic of', 'Iran')
df['Country'] = df['Country'].replace('Syrian Arab Republic', 'Syria')
df['Country'] = df['Country'].replace('Viet Nam', 'Vietnam')
df['Country'] = df['Country'].replace('Holy See (Vatican City State)', 'Vatican')
df['Country'] = df['Country'].replace('Serbia', 'Republic of Serbia')
df['Country'] = df['Country'].replace('Tanzania, United Republic of', 'United Republic of Tanzania')

- Merge two datasets by country

In [5]:
geometry_df = pd.merge(countries[['ADMIN', 'geometry']], df, left_on="ADMIN", right_on="Country", how="right")
geometry_df = geometry_df.drop(columns=['Country', 'Alpha-3 code'])
geometry_df = geometry_df.rename(columns={'ADMIN': 'Country'})
geometry_df.head()

,Country,geometry,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,United States of America,"MULTIPOLYGON (((-122.75302 48.99251, -122.6532...",ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],US,1.0,NaN,NaN,NaN,0.600,NaN
1,United States of America,"MULTIPOLYGON (((-122.75302 48.99251, -122.6532...",tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,United States of America,"MULTIPOLYGON (((-122.75302 48.99251, -122.6532...",tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",US,NaN,tt0068473,7.7,107673.0,10.010,7.300
3,United Kingdom,"MULTIPOLYGON (((-7.24710 55.06932, -7.25674 55...",tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",GB,NaN,tt0071853,8.2,534486.0,15.461,7.811
4,United Kingdom,"MULTIPOLYGON (((-7.24710 55.06932, -7.25674 55...",tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']",GB,NaN,tt0061578,7.7,72662.0,20.398,7.600


- Create new column for every unique genre

In [6]:
import ast
unique_genres = set()
for index, row in geometry_df.iterrows():
    genres_list = ast.literal_eval(row['genres'])
    unique_genres.update(genres_list)

for genre in unique_genres:
    geometry_df[genre] = geometry_df['genres'].str.contains(genre)

In [7]:
columns_to_keep = [col for col in geometry_df.columns if col not in ['geometry']]
no_geometry_df = geometry_df[columns_to_keep]
no_geometry_df.head()

,Country,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,...,european,animation,war,reality,sport,comedy,crime,drama,documentation,fantasy
0,United States of America,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],US,...,False,False,False,False,False,False,False,False,True,False
1,United States of America,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",US,...,False,False,False,False,False,False,True,True,False,False
2,United States of America,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",US,...,True,False,False,False,False,False,False,True,False,False
3,United Kingdom,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",GB,...,False,False,False,False,False,True,False,False,False,True
4,United Kingdom,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']",GB,...,False,False,True,False,False,False,False,False,False,False


In [8]:
no_geometry_df.to_csv("no_geometry_data.csv")

# Part 2: Data Visualizations

`geometry_df` is the dataframe that contains `geometry` for every country. It will be used for our data visualizations that are maps. `geometry_df` is a very big dataframe (> 4 GB), considering that it might take long to produce data visualizations that doesn't use geometry, `no_geometry_df` is the dataframe without `geometry` column. 